# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [7]:
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.interpret import ExplanationClient
from azureml.automl.runtime.onnx_convert import OnnxConverter
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from sklearn.model_selection import train_test_split
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn

print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [8]:
ws = Workspace.from_config()
ws.write_config(path='.azureml')

# choose a name for experiment
experiment_name = 'automl-classification-experiment'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AYU4BM3PA to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-135114
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-135114


In [9]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "cpu-cluster-1"
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

Found existing cluster


In [10]:
key = 'uciheartfailuredataset'
if key in ws.datasets.keys(): 
    dataset = ws.datasets[key] 
    print("Found dataset")
else:
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
    dataset = Dataset.Tabular.from_delimited_files(url)
    dataset = dataset.register(workspace=ws, name=key)

Found dataset


In [11]:
ds = dataset.to_pandas_dataframe()
train, test = train_test_split(ds, random_state=625, shuffle=True)

In [12]:
train.to_csv('train.csv', index=False)
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])
train = Dataset.Tabular.from_delimited_files(path = [(datastore,'train.csv')])

Uploading an estimated of 1 files
Target already exists. Skipping upload for train.csv
Uploaded 0 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [13]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_hours" : 0.3,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,
    "primary_metric": 'accuracy',
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    debug_log = 'automl_errors.log',
    compute_target=compute_target,
    task="classification",
    training_data= train,
    label_column_name="DEATH_EVENT",
    enable_onnx_compatible_models=True,
    n_cross_validations= 3,
    **automl_settings)

In [14]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config,show_output = False)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [15]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [10]:
%%time
remote_run.wait_for_completion()

CPU times: user 40.1 ms, sys: 0 ns, total: 40.1 ms
Wall time: 4.14 s


{'runId': 'AutoML_53a210fc-9c19-4f91-aa28-e98fbf4fd982',
 'target': 'cpu-cluster-1',
 'status': 'Completed',
 'startTimeUtc': '2021-01-17T19:49:35.320308Z',
 'endTimeUtc': '2021-01-17T20:14:32.830701Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'cpu-cluster-1',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-classification-experiment","subscription_id":"9a7511b8-150f-4a58-8528-3e7d50216c31","resource_group":"aml-quickstarts-135114","workspace_name":"quick-starts-ws-135114","region":"southcentralus","compute_target":"cpu-cluster-1","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":n

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.

## Retrieve the Best ONNX Model


In [16]:
best_run, onnx_model= remote_run.get_output(return_onnx_model=True)

In [17]:
#TODO: Save the best model
OnnxConverter.save_onnx_model(onnx_model, file_path="./automl_model.onnx")


In [18]:
best_run.get_properties()

{'runTemplate': 'automl_child',
 'pipeline_id': '5dfac790c5c209f98a1da2dc1c7fb76f0397324f',
 'pipeline_spec': '{"objects":[{"spec_class":"preproc","class_name":"MaxAbsScaler","module":"sklearn.preprocessing","param_args":[],"param_kwargs":{},"prepared_kwargs":{}},{"spec_class":"sklearn","class_name":"LightGBMClassifier","module":"automl.client.core.common.model_wrappers","param_args":[],"param_kwargs":{"min_data_in_leaf":20},"prepared_kwargs":{}}],"pipeline_id":"5dfac790c5c209f98a1da2dc1c7fb76f0397324f","module":"sklearn.pipeline","class_name":"Pipeline"}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '0',
 '_aml_system_scenario_identification': 'Remote.Child',
 '_azureml.ComputeTargetType': 'amlcompute',
 'ContentSnapshotId': 'c104bb85-1768-40d1-ad0f-3a05d25a2434',
 'ProcessInfoFile': 'azureml-logs/process_info.json',
 'ProcessStatusFile': 'azureml-logs/process_status.json',
 'run_template': 'automl_child',
 'run_preprocessor': 'MaxAbsScaler',
 'run_algorithm': '

In [19]:
best_run.get_metrics()

{'average_precision_score_weighted': 0.9242701299408628,
 'f1_score_macro': 0.83215264830915,
 'AUC_weighted': 0.9105514340731732,
 'AUC_micro': 0.9266697291886481,
 'recall_score_macro': 0.8206040336475118,
 'weighted_accuracy': 0.8919532125251112,
 'balanced_accuracy': 0.8206040336475118,
 'precision_score_macro': 0.8497868217331971,
 'average_precision_score_macro': 0.9057521503732796,
 'precision_score_micro': 0.8614414414414414,
 'f1_score_micro': 0.8614414414414414,
 'log_loss': 0.3930102701444301,
 'norm_macro_recall': 0.6412080672950238,
 'f1_score_weighted': 0.8588237135782609,
 'recall_score_weighted': 0.8614414414414414,
 'AUC_macro': 0.9105514340731732,
 'accuracy': 0.8614414414414414,
 'matthews_correlation': 0.6695313726094962,
 'recall_score_micro': 0.8614414414414414,
 'precision_score_weighted': 0.8607005358875601,
 'average_precision_score_micro': 0.9257810372323686,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_236ad473-eaa7-49b9-8324-8eaf91045017_0/

In [20]:
print(best_run)

Run(Experiment: automl-classification-experiment,
Id: AutoML_236ad473-eaa7-49b9-8324-8eaf91045017_0,
Type: azureml.scriptrun,
Status: Completed)


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [21]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')

In [22]:
description = 'AutoML Model'
tags = None
model = remote_run.register_model(model_name = model_name, description = description, tags = tags)
print(remote_run.model_id) 

AutoML236ad473e0


In [24]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script='inference/score.py')

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'type': "automl_classification"}, 
                                               description = 'Heart Data Service')

aci_service_name = 'automl-uciheart1'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

automl-uciheart1
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


## Enable Application Insitghts on The Deployed Service

In [25]:
aci_service.update(enable_app_insights=True)

## Service Details

In [26]:
print(f'\nservice state: {aci_service.state}\n')
print(f'scoring URI: \n{aci_service.scoring_uri}\n')
print(f'swagger URI: \n{aci_service.swagger_uri}\n')


service state: Healthy

scoring URI: 
http://df9f4240-1004-41a8-97fa-970cb972a48c.southcentralus.azurecontainer.io/score

swagger URI: 
http://df9f4240-1004-41a8-97fa-970cb972a48c.southcentralus.azurecontainer.io/swagger.json



## Test Service : 
TODO: In the cell below, send a request to the web service you deployed to test it.

In [29]:
import requests
import json

scoring_uri = aci_service.scoring_uri
# Two sets of data to score, so we get two results back
data = {"data":
        [
          { 
            "age": 45,
            "anaemia": 0,
            "creatinine_phosphokinase": 2413,
            "diabetes": 0,
            "ejection_fraction": 38,
            "high_blood_pressure": 0,
            "platelets": 140000,
            "serum_creatinine": 1.4,
            "serum_sodium": 140,
            "sex": 1,
            "smoking": 1,
            "time": 280
          },
          {
            "age": 50,
            "anaemia": 0,
            "creatinine_phosphokinase": 196,
            "diabetes": 0,
            "ejection_fraction": "45",
            "high_blood_pressure": 0,
            "platelets": 395000,
            "serum_creatinine": 1.6,
            "serum_sodium": 136,
            "sex":1,
            "smoking": 1,
            "time": 285
          },
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)
# Set the content type
headers = {'Content-Type': 'application/json'}
# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

{"result": [0, 0]}


TODO: In the cell below, print the logs of the web service and delete the service

In [30]:
aci_service.get_logs()

'2021-01-17T21:25:29,973597000+00:00 - iot-server/run \n2021-01-17T21:25:29,990956400+00:00 - rsyslog/run \n2021-01-17T21:25:29,998144000+00:00 - gunicorn/run \n2021-01-17T21:25:30,048517800+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

In [ ]:
aci_service.delete()
model.delete()
run.delete()
experiment.delete()
compute_target.delete()